### Set up

Library

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import json
import subprocess
import time
import threading
import psycopg2
import pandas as pd
import numpy as np
import getpass
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

e:\NhiAri\Code\GroupProject2024\groupproject2024\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#### postgresSQL

In [ ]:
postgres_url = getpass.getpass("Enter your postgresql url: ")

In [4]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

#### Gemini

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Query

Có 2 thành phần chính filter và recommen
Filter là các câu query vào SQL. Cách tính giá sao cho phù hợp hay thế nào nằm ở phần Filter, Recommen có công dụng là đề xuất cái tốt nhất

Prompt để trích xuất thông tin. Để đưa được vào câu prompt thì trước hết phải lấy ra các giá trị xuất hiện bên trong cột style, amenities. Dưới đây là mẫu.


#### Get features

##### Hotels

In [8]:
# amenities
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(amenities) AS unique_amenities
    FROM hotel;
""")

rows = cur.fetchall()

cur.close()
conn.close()

amenities_list = [row[0] for row in rows]
amenities_list_str = "\n    ".join(f'"{amenities_type}"' for amenities_type in amenities_list)
print(amenities_list_str)

"WiFi tại khu vực chung"
    "Money changer"
    "Bathtub"
    "Tiện nghi cho trẻ"
    "Quầy bar bên hồ bơi"
    "Heater"
    "Dù (ô) che nắng"
    "Đưa đón đến khu trượt tuyết (thu phí)"
    "Tiện nghi hội họp"
    "Sân quần vợt ngoài trời"
    "Trung tâm chăm sóc trẻ em"
    "Roll-in shower"
    "Vegetarian meal"
    "Hồ bơi"
    "Tủ lạnh (dùng chung)"
    "Máy photocopy"
    "Giữ trẻ"
    "Porter"
    "Bicycle storage"
    "Dịch vụ cho thuê xe đạp"
    "Conference room"
    "Dịch vụ trông trẻ có người giám hộ"
    "Grocery"
    "Giặt ủi"
    "Dịch vụ phòng 24 giờ"
    "Lò vi sóng"
    "Express check-out"
    "Đưa đón sân bay"
    "AC"
    "Dịch vụ phòng (có giới hạn thời gian)"
    "Bóng quần"
    "TV lounge"
    "A la carte lunch"
    "Bồn tắm nước nóng"
    "Gói cầu hôn lãng mạn"
    "Safety deposit box"
    "Vườn thú bán hoang dã"
    "Hồ bơi trẻ em"
    "Ghế dài tắm nắng"
    "Đưa đón đến trạm xe buýt (thu phí)"
    "Wifi (miễn phí)"
    "Thẩm mỹ viện"
    "Bữa trưa món tự chọn"

In [9]:
# style
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT style
    FROM hotel
    WHERE style IS NOT NULL;
""")

rows = cur.fetchall()

cur.close()
conn.close()
style_list = [row[0] for row in rows]
style_list_str = "\n    ".join(f'"{style}"' for style in style_list)
print(style_list_str)

"Business
"
    "Adventure 
"
    "Romantic 
"
    "Eco-friendly 
"
    "Business 
"
    "Wellness 
"
    "Family-friendly 
"
    "Cultural
"
    "Romantic
"
    "Beachfront 
"
    "Luxury
"
    "Boutique
"
    "Eco-friendly
"
    "Adventure
"
    "Family-friendly
"
    "Boutique 
"
    "Cultural 
"
    "Wellness
"
    "Luxury 
"


##### Attractions

In [10]:
# types
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(attraction_type) AS unique_attraction_type
    FROM touristattraction;
""")

rows = cur.fetchall()

cur.close()
conn.close()

att_type_list = [row[0] for row in rows]
att_type_list_str = "\n    ".join(f'"{att_type}"' for att_type in att_type_list)
print(att_type_list_str)

"Nhà hát và biểu diễn"
    "Viện bảo tàng lịch sử"
    "Thủy cung"
    "Khu vực đi dạo tham quan di tích lịch sử"
    "Trường đại học và trường học"
    "Quán bar và câu lạc bộ"
    "Khu vực đi dạo ngắm cảnh"
    "Viện bảo tàng nghệ thuật"
    "Vườn"
    "Di tích cổ"
    "Nhà thờ và nhà thờ lớn"
    "ATV và xe địa hình"
    "Đài kỷ niệm và tượng"
    "Cầu"
    "Chuyến tham quan văn hóa"
    "Xưởng vẽ và làm đồ gốm"
    "Núi"
    "Địa điểm giáo dục"
    "Khu liên hợp thể thao"
    "Buổi học và hội thảo"
    "Cửa hàng đồ cổ"
    "Sân gôn"
    "Triển lãm"
    "Đấu trường và sân vận động"
    "Phòng trưng bày nghệ thuật"
    "Điểm thu hút khách tham quan và thắng cảnh"
    "Địa điểm tâm linh"
    "Chợ hoa"
    "Cửa hàng của nhà máy"
    "Trung tâm nghệ thuật"
    "Quán bar rượu vang"
    "Căn cứ và doanh trại quân đội"
    "Địa điểm lịch sử"
    "Công viên nước"
    "Chuyến tham quan cà phê và trà"
    "Nhà hát"
    "Trung tâm trò chơi và giải trí"
    "Bảo t

##### Restaurant

In [11]:
# Type
# Establish the connection
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Set the search path to use the correct schema
cur.execute("SET search_path TO travel_database, public;")

# Query to extract distinct districts from the address composite type
cur.execute("""
    SELECT DISTINCT unnest(restaurant_type) AS unique_res_type
    FROM restaurant;
""")

# Fetch all rows
rows = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()

# Convert the rows into a list and format the output
res_type_list = [row[0] for row in rows]
res_type_list_str = "\n    ".join(f'"{res_type}"' for res_type in res_type_list)
print(res_type_list_str)

"Karaoke"
    "Café/Dessert"
    "Buffet"
    "Ăn vặt/vỉa hè"
    "Tiệc cưới/Hội nghị"
    "Quán ăn"
    "Tiệm bánh"
    "Ăn chay"
    "Nhà hàng"


In [12]:
# Suitable for
# Establish the connection
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Set the search path to use the correct schema
cur.execute("SET search_path TO travel_database, public;")

# Query to extract distinct districts from the address composite type
cur.execute("""
    SELECT DISTINCT unnest(suitable_for) AS unique_res_suit
    FROM restaurant;
""")

# Fetch all rows
rows = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()

# Convert the rows into a list and format the output
res_suit_list = [row[0] for row in rows]
res_suit_list_str = "\n    ".join(f'"{res_suit}"' for res_suit in res_suit_list)
print(res_suit_list_str)

"Uống bia - Nhậu"
    "Ăn gia đình"
    "Ăn chay"
    "Ăn Fastfood"
    "Đãi tiệc"
    "Tiếp khách"
    "Takeaway - Mang về"
    "Họp nhóm"
    "Ăn vặt"
    "Nghe nhạc"
    "Du lịch"
    "Ngắm cảnh"
    "Chụp hình - Quay phim"
    "BBQ - Món Nướng"
    "Tiệc ngoài trời"
    "Thư giãn"
    "Hẹn hò"
    "Buffet"


#### Prompt

thêm district

Dựa vào câu prompt này để lấy ra các thông tin yêu cầu về hotel, restaurant, TouristAttraction

##### Extract first JSON

In [13]:
travel_type_list = ["Food Tour", "Văn hóa", "Thư giãn", "Trải nghiệm"]
companion_list = ["friends", "family", "colleagues"]
transport_list = ["self-drive car", "motorbike", "bicycle", "public transport"]
city_list = ["Hà Nội"]
district_list = ["Ba Đình", "Hoàn Kiếm", "Tây Hồ", "Long Biên", "Cầu Giấy", "Đống Đa", "Hai Bà Trưng", "Hoàng Mai", "Thanh Xuân", "Nam Từ Liêm", "Bắc Từ Liêm", "Hà Đông", "Sơn Tây"]

In [14]:
template = """
You are an AI travel suggestion chatbot. Analyze the following travel request:

Request: "{travel_request}"

Extract general and specific requirements for Hotels, Restaurants, and Tourist Attractions, even if some are not explicitly mentioned. For each type, provide the following information:

**General Requirements:**
- Type: Extract the travel type mentioned in the request and translated it if it needed, must be one from this list: {travel_type_list} or return null if not specified or only ask for one of Hotels, Restaurants, or Tourist Attractions.
- Number_of_people: Extract the number of people or return null if not specified.
- Companions: Extract the companions mentioned in the request and translated it if it needed, must be one from this list: {companion_list} or return null if not specified.
- Transportation: Identify the transportation method mentioned in the request and translated, convert it if it needed, transportation must be one from this list: {transport_list} or return null if not specified.
- Time: Any specific dates or time ranges mentioned or return null if not specified.
- City: The mentioned city (without "city" or "province") and must be one from this list: {city_list} or else return null.
- District: The mentioned district (without "district") and must be one frin this list: {district_list} or else return null.
- Price_range: Specify as "low", "medium", or "high" based on the request.

**For Hotels, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Amenities: Only include amenities from this list if explicitly mentioned in the request: {amenities_list} or else return null. If in Transportation they give information about "car", add "Bãi đỗ xe" to the Amenities.
- Style: Only include styles from this list if explicitly mentioned in the request: {style_list} or else return null

**For Restaurants, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Restaurant_Type: From this list: {restaurant_type_list}
- Suitable_For: From this list: {suitable_for_list}

**For Tourist Attractions, also identify:**
- Requirements: A list of specific requirements or preferences mentioned.
- Attraction_Type: From this list: {attraction_type_list}

Return the result using the following JSON format:

```json
{{
  "General": {{
    "Type": "...",
    "Number_of_people": "...",
    "Companion": "...",
    "Transportation": "...",
    "Time": "...",
    "City": "...",
    "District": "...",
    "Price_range": "...",
    "
  }},
  "Hotel": {{
    "Requirements": ...,
    "Amenities": [...],
    "Style": "..."
  }},
  "Restaurant": {{
    "Requirements": ...,
    "Restaurant_Type": "...",
    "Suitable_For": "..."
  }},
  "TouristAttraction": {{
    "Attraction_Type": "..."
  }}
}}

```

Ensure the JSON is valid. Use null for any unspecified information.
After the JSON output, add a note in Vietnamese:

"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."
"""

In [15]:
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

##### Ask again if there's missing infor

format json due to type

ưu tiên hỏi time

In [48]:
ask_template = """
You are an AI travel suggestion chatbot. Analyze the following travel request:

Request: "{travel_output_json}"

### **Rules for generating questions:**
-**Generate questions only if both of these conditions are met:**
   1. The field that is **explicitly null** or have null value in the General section.
   2. The field is marked as *must ask question if this field is null, else not*.

-**IMPORTANT: Check ALL null fields in General section that are marked as *must ask question if this field is null, else not*:**
   - Type (if null)
   - Companion (if null)
   - Transportation (if null)
   - Time (if null)
   - Price_range (if null)

-**Strictly skip any field that already has a value different from `null` (non-null value), even if marked as *must ask question if this field is null, else not*.
-**Fields to Check for Questions:**
   **For General:**
   - Type: From this list: {travel_type_list} *must ask question if this field is null, else not*
   - Number_of_people: *must ask question if this field is null, else not*
   - Companion: From this list: {companion_list} *must ask question if this field is null, else not*
   - Transportation: From this list: {transport_list} *must ask question if this field is null, else not*
   - Time: *must ask question if this field is null, else not*
   - City: From this list: {city_list}. If `City` is not in the list, ask if the user wants to change it to one of the cities in the list.
   - District: From this list: {district_list}
   - Price_range: *must ask question if this field is null, else not*

   **For Hotels:**
   - Requirements: 
   - Amenities: From this list: {amenities_list}
   - Style: From this list: {style_list}

   **For Restaurants:**
   - Requirements: 
   - Restaurant_Type: From this list: {restaurant_type_list}
   - Suitable_For: From this list: {suitable_for_list}

   **For Tourist Attractions:**
   - Requirements: 
   - Attraction_Type: From this list: {attraction_type_list}

-**Ask only if specific fields are both strictly `null` and marked as *must ask question if this field is null, else not*. Strictly skip the question for any field that has a non-null value, regardless of whether it is marked with *must ask question if this field is null, else not*.**
-**If a field has a value different from `null`, skip the question.**
---

Analyze the JSON to find the fields that have null value. **Only ask questions for fields explicitly marked as *must ask question if this field is null, else not* and set to `null`.** If a populated field is not in the required list, ask if the user would like to change it to a valid option.

**Generate questions for null values in *must ask question if this field is null, else not* fields and if necessary, verify if the City needs adjustment:**
**Skip the question for any field that has a non-null value, regardless of whether it is marked with *must ask question if this field is null, else not*.**

---

###**Questions to generate for null fields - ENSURE ALL NULL FIELDS ARE CHECKED:**
1. If `"Type"` is null, ask:  
   **"Bạn muốn tìm loại hình du lịch nào? (Ví dụ: Food Tour, Văn hóa, Thư giãn, hoặc Trải nghiệm)"**

2. If `"Number_of_people"` is null, ask:  
   **"Bạn đi bao nhiêu người? (Ví dụ: 1, 2, hoặc nhóm lớn hơn)"**

3. If `"Companion"` is null, ask:  
   **"Bạn đi cùng ai? (Bạn bè, Gia đình, hoặc Đồng nghiệp)"**

4. If `"Transportation"` is null, ask:  
   **"Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)"**

5. If `"Time"` is null, ask:  
   **"Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)"**

6. If `"Price_range"` is null, ask:  
   **"Bạn muốn ngân sách cho chuyến đi này là bao nhiêu (thấp, trung bình, cao)?"**

7. If `"City"` is populated but not in the list {city_list}, ask:
   **"Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại {city_list}, liệu bạn có muốn thay đổi thành phố không?"**

### **Final Output:**
If all relevant fields with *must ask question if null* are filled, output:  
**"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."**

---

### **Example Output:**
If the provided JSON input has `"Transportation"` and `"Time"` as `null` and marked as *must ask question*, and `"City"` is "Đà Nẵng" (not in the list), the output will be:

```plaintext
Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)

Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)

Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại {city_list}, liệu bạn có muốn thay đổi thành phố không?

Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.
"""

In [49]:
ask_prompt = ChatPromptTemplate.from_template(ask_template)
ask_chain = ask_prompt | llm

prompt to update requirement json

In [57]:
updated_query = """
You are an AI travel suggestion chatbot. Analyze the following travel request:

Update request: "{update_travel_request}"

Extract general requirements from request while following these rules:
1. IMPORTANT: Preserve ALL non-null values from the initial request JSON
2. Only update fields that are null in the initial request OR if the city is explicitly changed
3. For new information, extract the following:

**General Requirements:**
- Type: Extract the travel type mentioned in the request and translated it if it needed, must be one from this list: {travel_type_list} or return null and ignore the question about Type if only ask for one of Hotels, Restaurants, or Tourist Attractions.
- Number_of_people: Extract the number of people.
- Companions: Extract the companions mentioned in the request and translated it if it needed, must be one from this list: {companion_list}.
- Transportation: Identify the transportation method mentioned in the request and translated, convert it if it needed, transportation must be one from this list: {transport_list}.
- Time: Any specific dates or time ranges mentioned.
- City: The mentioned city (without "city" or "province") and must be one from this list: {city_list}.
- Price_range: Specify as "low", "medium", or "high" based on the request.

Initial request: "{travel_output_json}"

Merge the initial request with any updates, prioritizing:
1. Keeping all non-null values from initial request
2. Only updating null fields or explicitly changed city
3. Using the following JSON format:

```json
{{
  "General": {{
    "Type": "...",
    "Number_of_people": "...",
    "Companion": "...",
    "Transportation": "...",
    "Time": "...",
    "City": "...",
    "District": "...",
    "Price_range": "...",
    "
  }},
  "Hotel": {{
    "Requirements": ...,
    "Amenities": [...],
    "Style": "..."
  }},
  "Restaurant": {{
    "Requirements": ...,
    "Restaurant_Type": "...",
    "Suitable_For": "..."
  }},
  "TouristAttraction": {{
    "Attraction_Type": "..."
  }}
}}

```

IMPORTANT: Before outputting, verify that all non-null values from the initial request are preserved unless explicitly changed in the update request.

Ensure the JSON is valid. Use null for any unspecified information.
After the JSON output, add a note in Vietnamese:

"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."
"""

<!-- **For Hotels:**
- Amenities: If in Transportation they give information about "car", add "Bãi đậu xe" to the Amenities. -->

In [58]:
update_prompt = ChatPromptTemplate.from_template(updated_query)
update_chain = update_prompt | llm

#### Query from user requests

##### User requirement

In [264]:
# Your input query
user_query = """
Gợi ý cho tôi một lộ trình du lịch tại Hà Nội với khách sạn sang trọng, có Bồn tắm, bể bơi và Mát-xa toàn thân, nhà hàng phục vụ món ăn truyền thống và một điểm tham quan nổi tiếng phù hợp cho trẻ em về đề tài lịch sử.
Tôi muốn biết thêm về các tiện nghi của khách sạn và phong cách của nhà hàng.
Chúng tôi đi 4 người.
"""

In [20]:
# Your input query
user_query = """
Gợi ý cho tôi một lộ trình du lịch tại Đà Nẵng với khách sạn sang trọng.
Chúng tôi đi 4 người.
"""

first response

In [25]:
def user_requires(chain, query, travel_type_list, companion_list, transport_list, city_list, district_list, 
                  amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str):
    response = chain.invoke({
        "travel_request": query,
        "travel_type_list": travel_type_list,
        "companion_list": companion_list,
        "transport_list": transport_list,
        "city_list": city_list,
        "district_list": district_list,
        "amenities_list": amenities_list_str,
        "style_list": style_list_str,
        "restaurant_type_list": res_type_list_str,
        "suitable_for_list": res_suit_list_str,
        "attraction_type_list": att_type_list_str
    })

    # Extract and parse the JSON response
    try:
        json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
        if json_match:
            result_dict = json.loads(json_match.group(0))
            
            # Print the JSON result
            print("Extracted JSON Result:")
            print(json.dumps(result_dict, indent=2, ensure_ascii=False))
            return result_dict
        else:
            print("No JSON object found in the response.")
            return None
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        print("Raw response:", response.content)
        return None

In [26]:
user_requires_respond = user_requires(chain, user_query, travel_type_list, companion_list, transport_list, city_list, district_list, amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str)

Extracted JSON Result:
{
  "General": {
    "Type": null,
    "Number_of_people": 4,
    "Companion": null,
    "Transportation": null,
    "Time": null,
    "City": "Đà Nẵng",
    "District": null,
    "Price_range": "high"
  },
  "Hotel": {
    "Requirements": "Khách sạn sang trọng",
    "Amenities": [],
    "Style": "Luxury"
  },
  "Restaurant": {
    "Requirements": null,
    "Restaurant_Type": null,
    "Suitable_For": null
  },
  "TouristAttraction": {
    "Requirements": null,
    "Attraction_Type": null
  }
}


ask again if missing values

In [52]:
def ask_user(ask_chain, response, travel_type_list, companion_list, transport_list, city_list, district_list, 
                  amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str):
    response1 = ask_chain.invoke({
        "travel_output_json": response,
        "travel_type_list": travel_type_list,
        "companion_list": companion_list,
        "transport_list": transport_list,
        "city_list": city_list,
        "district_list": district_list,
        "amenities_list": amenities_list_str,
        "style_list": style_list_str,
        "restaurant_type_list": res_type_list_str,
        "suitable_for_list": res_suit_list_str,
        "attraction_type_list": att_type_list_str
    })

    print("Cảm ơn bạn đã cung cấp thông tin! Tuy nhiên, tôi cần thêm một số thông tin để giúp bạn tốt hơn:")
    # print(response1.content)
    user_responses = {}

    # Process each line in the response content as a separate question
    questions = response1.content.splitlines()
    for question in questions:
        # Remove "plaintext:" prefix if it exists and trim whitespace
        question = question.replace("```plaintext", "").replace("```", "").strip()
        
        # Skip empty lines and avoid re-asking filled fields
        if not question or "Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào" in question:
            print(question)  # Print closing statement without asking for input
            continue
        
        # Ask the user for input and store the response
        user_input = input(f"{question} ")
        user_responses[question] = f"[{user_input.strip()}]"
        
    # Print the responses collected for review
    print("\nCollected User Responses:")
    for field, answer in user_responses.items():
        print(f"{field}: {answer}")
    
    print("Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.")
    return user_responses

In [51]:
response1 = ask_chain.invoke({
        "travel_output_json": user_requires_respond,
        "travel_type_list": travel_type_list,
        "companion_list": companion_list,
        "transport_list": transport_list,
        "city_list": city_list,
        "district_list": district_list,
        "amenities_list": amenities_list_str,
        "style_list": style_list_str,
        "restaurant_type_list": res_type_list_str,
        "suitable_for_list": res_suit_list_str,
        "attraction_type_list": att_type_list_str
})

print(response1.content)

```plaintext
Bạn muốn tìm loại hình du lịch nào? (Ví dụ: Food Tour, Văn hóa, Thư giãn, hoặc Trải nghiệm)

Bạn đi cùng ai? (Bạn bè, Gia đình, hoặc Đồng nghiệp)

Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)

Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)

Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại ['Hà Nội'], liệu bạn có muốn thay đổi thành phố không?

Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.
``` 



In [53]:
ask_again_respond = ask_user(ask_chain, user_requires_respond, travel_type_list, companion_list, transport_list, city_list, district_list, amenities_list_str, style_list_str, res_type_list_str, res_suit_list_str, att_type_list_str)

Cảm ơn bạn đã cung cấp thông tin! Tuy nhiên, tôi cần thêm một số thông tin để giúp bạn tốt hơn:






Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.


Collected User Responses:
Bạn muốn tìm loại hình du lịch nào? (Ví dụ: Food Tour, Văn hóa, Thư giãn, hoặc Trải nghiệm): [foodtour]
Bạn đi cùng ai? (Bạn bè, Gia đình, hoặc Đồng nghiệp): [bạn bè]
Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng): [ô tô]
Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian): [4 ngày]
Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại ['Hà Nội'], liệu bạn có muốn thay đổi thành phố không?: [hà nội]
Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.


update requirement json

In [59]:
def update_requires(update_chain, first_respond, travel_type_list, companion_list, transport_list, city_list, update_respond):
    response = update_chain.invoke({
        "update_travel_request": update_respond,
        "travel_type_list": travel_type_list,
        "companion_list": companion_list,
        "transport_list": transport_list,
        "city_list": city_list,
        "travel_output_json": first_respond
    })

    # Extract and parse the JSON response
    try:
        json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
        if json_match:
            result_dict = json.loads(json_match.group(0))
            
            # Print the JSON result
            print("Extracted JSON Result:")
            print(json.dumps(result_dict, indent=2, ensure_ascii=False))
            return result_dict
        else:
            print("No JSON object found in the response.")
            return None
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        print("Raw response:", response.content)
        return None

In [60]:
update_requires_respond = update_requires(update_chain, user_requires_respond, travel_type_list, companion_list, transport_list, city_list, ask_again_respond)

Extracted JSON Result:
{
  "General": {
    "Type": "Food Tour",
    "Number_of_people": 4,
    "Companion": "friends",
    "Transportation": "self-drive car",
    "Time": "4 days",
    "City": "Hà Nội",
    "District": null,
    "Price_range": "high"
  },
  "Hotel": {
    "Requirements": "Khách sạn sang trọng",
    "Amenities": [],
    "Style": "Luxury"
  },
  "Restaurant": {
    "Requirements": null,
    "Restaurant_Type": null,
    "Suitable_For": null
  },
  "TouristAttraction": {
    "Requirements": null,
    "Attraction_Type": null
  }
}


##### query from database

In [57]:
def build_sql_query_individual(table, requirements, general_requirements, schema="travel_database"):
    conditions = []
    joins = ""
    
    # Define the ID column based on the table
    id_column = "id"
    if table == "hotel":
        id_column = "hotel_id"
    elif table == "restaurant":
        id_column = "res_id"
    elif table == "touristattraction":
        id_column = "attraction_id"
    
    # Handle Price_range for each table with join and conditions
    if general_requirements.get("Price_range"):
        price_range = general_requirements["Price_range"]
        if table == "hotel":
            joins += f" JOIN {schema}.hotelprice ON {schema}.hotel.hotel_id = {schema}.hotelprice.hotel_id"
            if price_range == "low":
                conditions.append(f"{schema}.hotelprice.price < 500000")
            elif price_range == "medium":
                conditions.append(f"{schema}.hotelprice.price <= 2000000")
            elif price_range == "high":
                conditions.append(f"{schema}.hotelprice.price > 2000000")
        elif table == "restaurant":
            if price_range == "low":
                conditions.append(f"CAST({schema}.restaurant.price_range->>'max_price' AS INTEGER) < 200000")
            elif price_range == "medium":
                conditions.append(f"CAST({schema}.restaurant.price_range->>'max_price' AS INTEGER) <= 600000")
            elif price_range == "high":
                conditions.append(f"CAST({schema}.restaurant.price_range->>'min_price' AS INTEGER) >= 0")
        elif table == "touristattraction":
            joins += f" JOIN {schema}.attractionprice ON {schema}.touristattraction.attraction_id = {schema}.attractionprice.attraction_id"
            if price_range == "low":
                conditions.append(f"{schema}.attractionprice.price < 500000")
            elif price_range == "medium":
                conditions.append(f"{schema}.attractionprice.price < 1500000")
            elif price_range == "high":
                conditions.append(f"{schema}.attractionprice.price >= 0")
    
    if general_requirements.get("Transportation"):
        if general_requirements["Transportation"] == "self-drive car":
            if table == "restaurant":
                conditions.append(f"{schema}.restaurant.parking_available = TRUE")
            elif table == "hotel":
                conditions.append(f"('Bãi đậu xe' = ANY({schema}.hotel.amenities) OR 'Garage' = ANY({schema}.hotel.amenities))")
    # Process specific requirements for each table
    if table == "hotel":
        if requirements.get("Amenities"):
            amenities_condition = (" OR ".join([f"'{amenity}' = ANY({schema}.hotel.amenities)" for amenity in requirements["Amenities"]]))
            conditions.append(f"({amenities_condition})")
        if requirements.get("Style") and requirements["Style"] != "none":
            conditions.append(f"{schema}.hotel.style LIKE '{requirements['Style']}%'")
    elif table == "restaurant":
        if requirements.get("Restaurant_Type"):
            conditions.append(f"'{requirements['Restaurant_Type']}' = ANY({schema}.restaurant.restaurant_type)")
        if requirements.get("Suitable_For"):
            conditions.append(f"'{requirements['Suitable_For']}' = ANY({schema}.restaurant.suitable_for)")
    elif table == "touristattraction":
        if requirements.get("Attraction_Type"):
            conditions.append(f"'{requirements['Attraction_Type']}' = ANY({schema}.touristattraction.attraction_type)")

    # Build the WHERE clause and complete query
    where_clause = " AND ".join(conditions)
    query = (
        f"SELECT DISTINCT {schema}.{table}.{id_column}, {schema}.{table}.name "
        f"FROM {schema}.{table} {joins} WHERE {where_clause};"
        if conditions else
        f"SELECT {schema}.{table}.{id_column}, {schema}.{table}.name FROM {schema}.{table};"
    )

    return query

In [58]:
def fetch_locations(query):
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()

    # Thiết lập search_path
    cur.execute("""SET search_path TO travel_database, public;""")
    cur.execute(query)

    # Lấy tất cả các kết quả
    results = cur.fetchall()

    return results

    # Đóng kết nối
    cur.close()
    conn.close()

In [59]:
general_requirements = update_requires_respond.get("General", {})
hotel_requirements = update_requires_respond.get("Hotel", {})
restaurant_requirements = update_requires_respond.get("Restaurant", {})
attraction_requirements = update_requires_respond.get("TouristAttraction", {})

# Build SQL queries
hotel_query_indi = build_sql_query_individual("hotel", hotel_requirements, general_requirements)
restaurant_query_indi = build_sql_query_individual("restaurant", restaurant_requirements, general_requirements)
attraction_query_indi = build_sql_query_individual("touristattraction", attraction_requirements, general_requirements)

print("Hotel Query:", hotel_query_indi)
print("\nRestaurant Query:", restaurant_query_indi)
print("\nAttraction Query:", attraction_query_indi)

Hotel Query: SELECT DISTINCT travel_database.hotel.hotel_id, travel_database.hotel.name FROM travel_database.hotel  JOIN travel_database.hotelprice ON travel_database.hotel.hotel_id = travel_database.hotelprice.hotel_id WHERE travel_database.hotelprice.price > 2000000 AND ('Bãi đậu xe' = ANY(travel_database.hotel.amenities) OR 'Garage' = ANY(travel_database.hotel.amenities)) AND ('Bathtub' = ANY(travel_database.hotel.amenities) OR 'Hồ bơi' = ANY(travel_database.hotel.amenities) OR 'Mát-xa' = ANY(travel_database.hotel.amenities)) AND travel_database.hotel.style LIKE 'Luxury%';

Restaurant Query: SELECT DISTINCT travel_database.restaurant.res_id, travel_database.restaurant.name FROM travel_database.restaurant  WHERE CAST(travel_database.restaurant.price_range->>'min_price' AS INTEGER) >= 0 AND travel_database.restaurant.parking_available = TRUE AND 'Nhà hàng' = ANY(travel_database.restaurant.restaurant_type) AND 'Ăn gia đình' = ANY(travel_database.restaurant.suitable_for);

Attraction Qu

In [60]:
general_requirements = update_requires_respond.get("General", {})
hotel_requirements = update_requires_respond.get("Hotel", {})
restaurant_requirements = update_requires_respond.get("Restaurant", {})
attraction_requirements = update_requires_respond.get("TouristAttraction", {})

# Build SQL queries
hotel_query_indi = build_sql_query_individual("hotel", hotel_requirements, general_requirements)
restaurant_query_indi = build_sql_query_individual("restaurant", restaurant_requirements, general_requirements)
attraction_query_indi = build_sql_query_individual("touristattraction", attraction_requirements, general_requirements)

hotel_locations = fetch_locations(hotel_query_indi)
restaurant_locations = fetch_locations(restaurant_query_indi)
attraction_locations = fetch_locations(attraction_query_indi)

# In ra danh sách các địa điểm để kiểm tra
# Print Hotel Locations
print("Hotel Locations:")
for location in hotel_locations:
    print(f"    id: {location[0]}, name: {location[1]}")

# Print Restaurant Locations
print("\nRestaurant Locations:")
for location in restaurant_locations:
    print(f"    id: {location[0]}, name: {location[1]}")

# Print Tourist Attraction Locations
print("\nTourist Attraction Locations:")
for location in attraction_locations:
    print(f"    id: {location[0]}, name: {location[1]}")

Hotel Locations:
    id: 7, name: Khách sạn Melia Hà Nội
    id: 8, name: Grand K Hotel Suites Hanoi
    id: 9, name: Muong Thanh Hanoi Centre Hotel
    id: 10, name: Lotte Hotel Hanoi
    id: 17, name: Super Hotel Candle
    id: 27, name: L7 West Lake Hanoi By Lotte
    id: 44, name: Army Hotel
    id: 47, name: InterContinental Hotels HANOI LANDMARK72, an IHG Hotel
    id: 63, name: Pan Pacific Hanoi
    id: 80, name: Grandiose Hotel & Spa
    id: 83, name: Hotel de l'Opera Hanoi - Mgallery
    id: 90, name: Somerset Hoa Binh Hanoi
    id: 109, name: Novotel Hanoi Thai Ha
    id: 123, name: Sen Grand Hotel & Spa
    id: 134, name: Khách sạn Elegant Suites Westlake
    id: 150, name: Fraser Residence Hanoi
    id: 205, name: Hôtel du Parc Hanoï
    id: 238, name: Khách sạn Hồng Ngọc Dynastie
    id: 245, name: WEST LAKE 254D HOTEL & RESIDENCE
    id: 312, name: Roygent Parks Hanoi
    id: 333, name: Wyndham Sky Lake Resort & Villas
    id: 334, name: InterContinental Hotels HANOI WEST

### Other code (not related to the main query)

#### Test query

In [61]:
# import pandas as pd
# from sqlalchemy import create_engine, text

# # Connect to the PostgreSQL database
# engine = create_engine(postgres_url)
# connection = engine.connect()

# # Load the hotel data into a DataFrame
# df = pd.read_sql("SELECT hotel_id, style FROM travel_database.hotel", connection)

# # Clean the style column: remove extra spaces and standardize casing
# df['style'] = df['style'].str.strip().str.title()

# # Verify cleaning
# print(df['style'].unique())  # See unique values to confirm cleaning

# # Update the cleaned data back to the database
# for index, row in df.iterrows():
#     update_query = text(f"""
#         UPDATE travel_database.hotel
#         SET style = :style
#         WHERE hotel_id = :hotel_id
#     """)
#     connection.execute(update_query, {"style": row['style'], "hotel_id": row['hotel_id']})

# # Close the connection
# connection.close()

In [62]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Thiết lập search_path
cur.execute("""SET search_path TO travel_database, public;""")

# Tạo câu truy vấn SQL
sql = """
    SELECT DISTINCT style
    FROM travel_database.hotel
;
"""

sql2= """
    SELECT DISTINCT travel_database.hotel.hotel_id, travel_database.hotel.name, travel_database.hotel.style, travel_database.hotel.amenities 
    FROM travel_database.hotel  
    JOIN travel_database.hotelprice 
    ON travel_database.hotel.hotel_id = travel_database.hotelprice.hotel_id 
    WHERE travel_database.hotelprice.price > 2000000 
        AND ('Bathtub' = ANY(travel_database.hotel.amenities) 
        OR 'Hồ bơi' = ANY(travel_database.hotel.amenities) 
        OR 'Mát-xa' = ANY(travel_database.hotel.amenities)) 
    AND travel_database.hotel.style LIKE 'Luxury%';
"""

# Thực thi câu truy vấn
cur.execute(sql2)

# Lấy tất cả các kết quả
attractions = cur.fetchall()

# In ra tên các attraction
for attraction in attractions:
    print(f"id: {attraction[0]} - name: {attraction[1]} - style: {attraction[2]} - amenities: {attraction[3]}\n")
    # print(f"{attraction[0]}")
# Đóng kết nối
cur.close()
conn.close()


id: 7 - name: Khách sạn Melia Hà Nội - style: Luxury
 - amenities: ['Quầy bar', 'Câu lạc bộ đêm', 'Quầy bar bên hồ bơi', 'Quầy phục vụ đồ ăn nhẹ', 'Bữa sáng', 'Bữa sáng (thu phí)', 'Nhà hàng có máy lạnh', 'Tiệm cà phê', 'Quầy bar', 'Bữa tối với thực đơn chọn sẵn', 'Bữa sáng và bữa tối', 'Đồ ăn nhẹ', 'Bữa sáng và bữa trưa', 'Bữa sáng cho người dậy sớm', 'Bữa sáng phục vụ món nóng', 'Bữa sáng phục vụ tại bàn', 'Bữa sáng muộn', 'Phục vụ món chay', 'Bữa sáng món tự chọn', 'Bữa sáng với thực đơn gọi món', 'Bữa sáng kiểu Âu', 'Bữa trưa món tự chọn', 'Bữa trưa với thực đơn gọi món', 'Bữa trưa với thực đơn chọn sẵn', 'Bữa tối món tự chọn', 'Bữa tối với thực đơn gọi món', 'Tiệc liên hoan', 'Dịch vụ concierge/hỗ trợ khách', 'Dịch vụ giặt ủi', 'Dịch vụ hỗ trợ đặt Tour', 'Dịch vụ lưu trữ/bảo quản hành lý', 'Nhân viên gác cổng', 'Nhật báo tại sảnh', 'Đội ngũ nhân viên đa ngôn ngữ', 'Dịch vụ tiệc cưới', 'Dịch vụ thu đổi ngoại tệ', 'Người gác cửa', 'Điện thoại di động', 'Bảo vệ 24 giờ', 'Nhân viên xá

In [63]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Thiết lập search_path
cur.execute("""SET search_path TO travel_database, public;""")

# Tạo câu truy vấn SQL
sql = """
    SELECT DISTINCT travel_database.touristattraction.attraction_id, travel_database.touristattraction.name, travel_database.touristattraction.attraction_type
    FROM travel_database.touristattraction  
    JOIN travel_database.attractionprice 
        ON travel_database.touristattraction.attraction_id = travel_database.attractionprice.attraction_id 
    WHERE (travel_database.attractionprice.price > 0 OR travel_database.attractionprice.price = 0)
        AND 'Viện bảo tàng lịch sử' = ANY(travel_database.touristattraction.attraction_type);
"""

# Thực thi câu truy vấn
cur.execute(sql)

# Lấy tất cả các kết quả
attractions = cur.fetchall()

# In ra tên các attraction
for attraction in attractions:
    print(f"id: {attraction[0]} - name: {attraction[1]} - type: {attraction[2]}")

# Đóng kết nối
cur.close()
conn.close()

id: 2 - name: Bảo tàng Dân tộc học - type: ['Viện bảo tàng lịch sử']
id: 7 - name: Bảo Tàng Hồ Chí Minh - type: ['Viện bảo tàng lịch sử']
id: 9 - name: Bảo Tàng Lịch Sử Quốc Gia - type: ['Viện bảo tàng lịch sử']
id: 10 - name: Ngôi Nhà Di Sản - type: ['Viện bảo tàng lịch sử']
id: 19 - name: Bảo Tàng Công An Nhân Dân Việt Nam - type: ['Viện bảo tàng lịch sử']
id: 21 - name: Bảo tàng Cách mạng Việt Nam - type: ['Viện bảo tàng lịch sử']
id: 22 - name: Bảo tàng Nguyễn Văn Huyên - type: ['Viện bảo tàng lịch sử']
id: 25 - name: Trung Tâm Giao Lưu Văn Hoá Phố Cổ - type: ['Viện bảo tàng lịch sử']
id: 27 - name: Bảo Tàng Đường Hồ Chí Minh - type: ['Viện bảo tàng lịch sử']


In [64]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Thiết lập search_path
cur.execute("""SET search_path TO travel_database, public;""")

# Tạo câu truy vấn SQL
sql = """
    SELECT DISTINCT travel_database.touristattraction.attraction_id, travel_database.touristattraction.name, travel_database.touristattraction.attraction_type
    FROM travel_database.touristattraction  
    JOIN travel_database.attractionprice 
        ON travel_database.touristattraction.attraction_id = travel_database.attractionprice.attraction_id 
    WHERE (travel_database.attractionprice.price > 0 OR travel_database.attractionprice.price = 0)
        AND 'Viện bảo tàng lịch sử' = ANY(travel_database.touristattraction.attraction_type);
"""

# Thực thi câu truy vấn
cur.execute(sql)

# Lấy tất cả các kết quả
attractions = cur.fetchall()

# In ra tên các attraction
for attraction in attractions:
    print(f"id: {attraction[0]} - name: {attraction[1]} - type: {attraction[2]}")

# Đóng kết nối
cur.close()
conn.close()

id: 2 - name: Bảo tàng Dân tộc học - type: ['Viện bảo tàng lịch sử']
id: 7 - name: Bảo Tàng Hồ Chí Minh - type: ['Viện bảo tàng lịch sử']
id: 9 - name: Bảo Tàng Lịch Sử Quốc Gia - type: ['Viện bảo tàng lịch sử']
id: 10 - name: Ngôi Nhà Di Sản - type: ['Viện bảo tàng lịch sử']
id: 19 - name: Bảo Tàng Công An Nhân Dân Việt Nam - type: ['Viện bảo tàng lịch sử']
id: 21 - name: Bảo tàng Cách mạng Việt Nam - type: ['Viện bảo tàng lịch sử']
id: 22 - name: Bảo tàng Nguyễn Văn Huyên - type: ['Viện bảo tàng lịch sử']
id: 25 - name: Trung Tâm Giao Lưu Văn Hoá Phố Cổ - type: ['Viện bảo tàng lịch sử']
id: 27 - name: Bảo Tàng Đường Hồ Chí Minh - type: ['Viện bảo tàng lịch sử']


In [65]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Thiết lập search_path
cur.execute("""SET search_path TO travel_database, public;""")

# Tạo câu truy vấn SQL
sql = """
    SELECT DISTINCT travel_database.restaurant.res_id, travel_database.restaurant.name, travel_database.restaurant.restaurant_type, travel_database.restaurant.suitable_for, CAST(travel_database.restaurant.price_range->>'min_price' AS INTEGER), travel_database.restaurant.parking_available
    FROM travel_database.restaurant  
    WHERE CAST(travel_database.restaurant.price_range->>'min_price' AS INTEGER) >= 0 
    AND travel_database.restaurant.parking_available = TRUE
    AND 'Nhà hàng' = ANY(travel_database.restaurant.restaurant_type) 
    AND 'Ăn gia đình' = ANY(travel_database.restaurant.suitable_for);
    """

# Thực thi câu truy vấn
cur.execute(sql)

# Lấy tất cả các kết quả
attractions = cur.fetchall()

# In ra tên các attraction
for attraction in attractions:
    print(f"id: {attraction[0]} - name: {attraction[1]} - type: {attraction[2]} - suitable: {attraction[3]} - price: {attraction[4]} - parking: {attraction[5]}")

# Đóng kết nối
cur.close()
conn.close()

id: 2 - name: Buffet Sen - Khu Văn Hoá Ẩm Thực Sen Tây Hồ - type: ['Nhà hàng'] - suitable: ['Đãi tiệc', 'Ăn gia đình', 'Hẹn hò', 'Họp nhóm', 'Tiếp khách', 'Ăn chay', 'Buffet', 'Tiệc ngoài trời', 'Ngắm cảnh', 'Nghe nhạc'] - price: 38000 - parking: True
id: 6 - name: Maison Vie - Nhà Hàng Kiểu Pháp - type: ['Nhà hàng'] - suitable: ['Ăn gia đình', 'Hẹn hò', 'Uống bia - Nhậu', 'Họp nhóm', 'Tiếp khách'] - price: 250000 - parking: True
id: 8 - name: Jacksons Steakhouse Hanoi - type: ['Nhà hàng'] - suitable: ['Đãi tiệc', 'Ăn gia đình', 'Hẹn hò', 'Họp nhóm', 'Tiếp khách'] - price: 300000 - parking: True
id: 14 - name: Salmonoid - Món Âu - Cao Bá Quát - type: ['Nhà hàng'] - suitable: ['Ăn gia đình', 'Hẹn hò', 'Họp nhóm', 'Tiếp khách'] - price: 200000 - parking: True
id: 15 - name: Jacksons Steakhouse Hanoi - type: ['Nhà hàng'] - suitable: ['Đãi tiệc', 'Ăn gia đình', 'Hẹn hò', 'Họp nhóm', 'Tiếp khách'] - price: 300000 - parking: True
id: 16 - name: Grille6 - Salad, Steak & Pasta - Hào Nam - type

In [66]:
def get_embedding(text):
    """Trả về embedding của văn bản sử dụng mô hình SentenceTransformer."""
    embedding = model.encode(text)
    return embedding.tolist()

#### Others way to query (not finish yet)

In [ ]:
# def build_sql_query(table, requirements, schema="travel_database"):
#     conditions = []
#     joins = ""
    
#     # Define the ID column based on the table
#     id_column = "id"
#     if table == "hotel":
#         id_column = "hotel_id"
#     elif table == "restaurant":
#         id_column = "res_id"
#     elif table == "touristattraction":
#         id_column = "attraction_id"
    
#     # Correctly access Price_range
#     price_range = requirements.get("General", {}).get("Price_range")
    
#     # Apply Price_range filters with JOIN if applicable
#     if price_range:
#         if table == "hotel":
#             joins += f" JOIN {schema}.hotelprice ON {schema}.hotel.hotel_id = {schema}.hotelprice.hotel_id"
#             if price_range == "low":
#                 conditions.append(f"{schema}.hotelprice.price < 500000")
#             elif price_range == "medium":
#                 conditions.append(f"{schema}.hotelprice.price BETWEEN 500000 AND 2000000")
#             elif price_range == "high":
#                 conditions.append(f"{schema}.hotelprice.price > 2000000")
#         elif table == "restaurant":
#             if price_range == "low":
#                 conditions.append(f"CAST({schema}.restaurant.price_range->>'max' AS INTEGER) < 200000")
#             elif price_range == "medium":
#                 conditions.append(f"CAST({schema}.restaurant.price_range->>'min' AS INTEGER) >= 200000 AND CAST({schema}.restaurant.price_range->>'max' AS INTEGER) <= 600000")
#             elif price_range == "high":
#                 conditions.append(f"CAST({schema}.restaurant.price_range->>'min' AS INTEGER) > 600000")
#         elif table == "touristattraction":
#             joins += f" JOIN {schema}.attractionprice ON {schema}.touristattraction.attraction_id = {schema}.attractionprice.attraction_id"
#             if price_range == "low":
#                 conditions.append(f"{schema}.attractionprice.price < 500000")
#             elif price_range == "medium":
#                 conditions.append(f"{schema}.attractionprice.price BETWEEN 500000 AND 1500000")
#             elif price_range == "high":
#                 conditions.append(f"{schema}.attractionprice.price > 1500000")

#     # Process specific requirements for each table
#     table_requirements = requirements.get(table.capitalize(), {})
    
#     if table == "hotel":
#         if table_requirements.get("Amenities"):
#             amenities_condition = " OR ".join([f"'{amenity}' = ANY({schema}.hotel.amenities)" for amenity in table_requirements["Amenities"]])
#             conditions.append(f"({amenities_condition})")
#         if table_requirements.get("Style") and table_requirements["Style"] != "none":
#             conditions.append(f"{schema}.hotel.style = '{table_requirements['Style']}'")
#     elif table == "restaurant":
#         if table_requirements.get("Restaurant_Type"):
#             conditions.append(f"'{table_requirements['Restaurant_Type']}' = ANY({schema}.restaurant.restaurant_type)")
#         if table_requirements.get("Suitable_For"):
#             conditions.append(f"'{table_requirements['Suitable_For']}' = ANY({schema}.restaurant.suitable_for)")
#     elif table == "touristattraction":
#         if table_requirements.get("Attraction_Type"):
#             conditions.append(f"'{table_requirements['Attraction_Type']}' = ANY({schema}.touristattraction.attraction_type)")

#     # Debugging output for conditions
#     # print("Conditions:", conditions)  # This can be commented out or logged

#     # Build the WHERE clause and complete query
#     where_clause = " AND ".join(conditions)
#     query = (
#         f"SELECT {schema}.{table}.{id_column}, {schema}.{table}.name "
#         f"FROM {schema}.{table} {joins} WHERE {where_clause};"
#         if conditions else
#         f"SELECT {schema}.{table}.{id_column}, {schema}.{table}.name FROM {schema}.{table};"
#     )

#     return query

In [ ]:
# hotel_query = build_sql_query("hotel", update_requires_respond)
# restaurant_query = build_sql_query("restaurant", update_requires_respond)
# attraction_query = build_sql_query("touristattraction", update_requires_respond)

# hotel_locations = fetch_locations(hotel_query)
# restaurant_locations = fetch_locations(restaurant_query)
# attraction_locations = fetch_locations(attraction_query)

# # In ra danh sách các địa điểm để kiểm tra
# print("Hotel Locations:", hotel_locations)
# print("Restaurant Locations:", restaurant_locations)
# print("Tourist Attraction Locations:", attraction_locations)